In [ ]:
!pip install firebase-admin pandas numpy matplotlib seaborn scikit-learn pyarrow joblib

In [ ]:
import os
import json
import time
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import firebase_admin
from firebase_admin import credentials, firestore
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KDTree
import threading
from joblib import Parallel, delayed
import multiprocessing
from firebase_admin import storage
from concurrent.futures import ThreadPoolExecutor, as_completed


# Configuration & Initialization
FIREBASE_KEY_PATH = "/content/firebase-key.json"
cred = credentials.Certificate(FIREBASE_KEY_PATH)
firebase_admin.initialize_app(cred, {
    'storageBucket': 'agrointel-dd8f6.firebasestorage.app'
})
db = firestore.client()
bucket = storage.bucket()

# Constants
UPLOAD_INTERVAL_SEC = 3600  # 10 minutes for testing (set to 3600 for 1 hour in production)
SNAPSHOT_COMPUTE_INTERVAL_SEC = 60  # Compute snapshot every 60 seconds
MINIMAL_LOOP_SLEEP = 0.1  # Small sleep to prevent CPU overuse
INITIAL_MIN_ENTRIES = 50
MAX_X_COORD = 1296
MAX_Y_COORD = 960
SMOOTHING_WINDOW = 5
PIXELS_TO_METERS = 100
OUTPUT_DIR = 'plots'
BATCH_SIZE = 500
VALID_IDS_FILE = 'valid_ids.json'
PROCESSED_DATA_FILE = 'processed_cows.parquet'
PROCESSED_TIMESTAMPS_FILE = 'processed_timestamps.json'
HISTORICAL_METRICS_FILE = 'historical_metrics.json'
MIN_NET_DISPLACEMENT = 0.05  # meters
STOP_SPEED_THRESHOLD = 0.01  # m/s

# Ensure output directories exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Globals
data_queue = []
stop_flag = False
processed_timestamps = set()
listeners = []
last_analysis_time = None
last_upload_time = None
current_df = pd.DataFrame()
historical_df = pd.DataFrame()
historical_metrics_summary = {}
latest_snapshot = None
data_lock = threading.Lock()
snapshot_compute_lock = threading.Lock()
last_total_distances = {}  # To track last known total distances per cow
last_processed_timestamps = {}  # To track last processed timestamp per cow

# Helper Functions
def init_processed_timestamps():
    global processed_timestamps, last_processed_timestamps
    if os.path.exists(PROCESSED_TIMESTAMPS_FILE):
        with open(PROCESSED_TIMESTAMPS_FILE, 'r') as f:
            processed_timestamps = set(json.load(f))
    elif os.path.exists(PROCESSED_DATA_FILE):
        df = pd.read_parquet(PROCESSED_DATA_FILE)
        if not df.empty and 'time' in df.columns:
            df['timestamp_str'] = df['time'].dt.strftime('%Y-%m-%dT%H:%M:%SZ')
            processed_timestamps = set(f"{row['id']}_{row['timestamp_str']}" for _, row in df.iterrows())
            last_processed_timestamps.update(
                df.groupby('id')['time'].max().to_dict()
            )
            save_processed_timestamps()
    else:
        print("No processed timestamps found, starting fresh")

def save_processed_timestamps():
    try:
        with open(PROCESSED_TIMESTAMPS_FILE, 'w') as f:
            json.dump(list(processed_timestamps), f)
    except Exception as e:
        print(f"Error saving processed timestamps: {e}")

def init_historical_metrics():
    global historical_metrics_summary
    if os.path.exists(HISTORICAL_METRICS_FILE):
        with open(HISTORICAL_METRICS_FILE, 'r') as f:
            historical_metrics_summary = json.load(f)
    else:
        print("No historical metrics found, starting fresh")
        historical_metrics_summary = {}

def save_historical_metrics():
    try:
        serializable_metrics = {}
        for cow_id, hourly_metrics in historical_metrics_summary.items():
            serializable_metrics[cow_id] = {}
            for hour_str, metrics in hourly_metrics.items():
                serializable_metrics[cow_id][hour_str] = {}
                for key, value in metrics.items():
                    if isinstance(value, (np.floating, np.integer)):
                        serializable_metrics[cow_id][hour_str][key] = float(value) if isinstance(value, np.floating) else int(value)
                    elif pd.isna(value):
                        serializable_metrics[cow_id][hour_str][key] = None
                    else:
                        serializable_metrics[cow_id][hour_str][key] = value
        with open(HISTORICAL_METRICS_FILE, 'w') as f:
            json.dump(serializable_metrics, f)
    except Exception as e:
        print(f"Error saving historical metrics: {e}")

def clear_plots_folder():
    for fn in os.listdir(OUTPUT_DIR):
        if fn.endswith('.png'):
            try:
                os.remove(os.path.join(OUTPUT_DIR, fn))
            except OSError:
                pass

def fetch_initial_data():
    records = []
    valid_ids = set()
    try:
        for cow_doc in db.collection('cows').stream():
            cow_id = cow_doc.id
            coord_count = 0
            coords_ref = db.collection('cows').document(cow_id).collection('coordinates')
            cow_records = []
            for coord_doc in coords_ref.stream():
                coord = coord_doc.to_dict()
                ts = coord.get('timestamp')
                if not ts:
                    continue
                eid = f"{cow_id}_{ts}"
                if eid in processed_timestamps:
                    continue
                coord['id'] = cow_id
                coord['time_in_zone'] = cow_doc.to_dict().get('time_in_zone', {})
                cow_records.append(coord)
                coord_count += 1
            if coord_count >= INITIAL_MIN_ENTRIES:
                valid_ids.add(cow_id)
                records.extend(cow_records)
        print(f"Fetched {len(records)} initial coordinate records")
        print(f"Identified {len(valid_ids)} valid cow IDs with at least {INITIAL_MIN_ENTRIES} entries")
    except Exception as e:
        print(f"Error querying cows collection: {e}")
    return pd.DataFrame(records), valid_ids

def update_valid_cow_ids():
    valid_ids = set()
    try:
        for cow_doc in db.collection('cows').stream():
            cow_id = cow_doc.id
            coords_ref = db.collection('cows').document(cow_id).collection('coordinates')
            count = sum(1 for _ in coords_ref.stream())
            if count >= INITIAL_MIN_ENTRIES:
                valid_ids.add(cow_id)
        save_valid_ids(valid_ids)
    except Exception as e:
        print(f"Error updating valid cow IDs: {e}")
    return valid_ids

def preprocess_data(df):
    if df.empty:
        print("No data to preprocess")
        return df
    df = df.copy()
    try:
        df['time'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%dT%H:%M:%SZ', utc=True, errors='coerce')
    except Exception as e:
        print(f"Error parsing timestamps: {e}")
        df['time'] = pd.to_datetime(df['timestamp'], errors='coerce')
    df = df.dropna(subset=['time'])
    df['x'] = df['coordinates'].apply(lambda c: c.get('x', np.nan) if isinstance(c, dict) else np.nan)
    df['y'] = df['coordinates'].apply(lambda c: c.get('y', np.nan) if isinstance(c, dict) else np.nan)
    df = df[(df['x'].between(0, MAX_X_COORD)) & (df['y'].between(0, MAX_Y_COORD))]
    if df.empty:
        print("All rows filtered out after preprocessing")
        return df
    df['id'] = df['id'].astype(str)
    df = df.drop_duplicates(subset=['id', 'time'])
    df.sort_values(['id', 'time'], inplace=True)
    df['x_smooth'] = df.groupby('id')['x'].transform(lambda s: s.rolling(window=min(SMOOTHING_WINDOW, len(s)), min_periods=1).mean())
    df['y_smooth'] = df.groupby('id')['y'].transform(lambda s: s.rolling(window=min(SMOOTHING_WINDOW, len(s)), min_periods=1).mean())
    df['prev_x'] = df.groupby('id')['x_smooth'].shift(1)
    df['prev_y'] = df.groupby('id')['y_smooth'].shift(1)
    df['distance_pixels'] = np.sqrt((df['x_smooth'] - df['prev_x'])**2 + (df['y_smooth'] - df['prev_y'])**2)
    df['distance_meters'] = (df['distance_pixels'] / PIXELS_TO_METERS).fillna(0)
    df['time_diff'] = df.groupby('id')['time'].diff().dt.total_seconds().fillna(0)
    df['speed'] = (df['distance_meters'] / df['time_diff']).replace([np.inf, -np.inf], 0).fillna(0)
    df['zone'] = df.get('zone', 'Null Zone').fillna('Null Zone')
    df['prev_zone'] = df.groupby('id')['zone'].shift(1)
    df['transition'] = (df['zone'] != df['prev_zone']) & df['prev_zone'].notna()
    df['x_norm'] = df['x'] / MAX_X_COORD
    df['y_norm'] = df['y'] / MAX_Y_COORD
    return df

def save_processed_data(df):
    if df.empty:
        return
    if os.path.exists(PROCESSED_DATA_FILE):
        existing = pd.read_parquet(PROCESSED_DATA_FILE)
        df = pd.concat([existing, df]).drop_duplicates(subset=['id', 'time'])
    df.to_parquet(PROCESSED_DATA_FILE)

def load_valid_ids():
    valid_ids = set(json.load(open(VALID_IDS_FILE))) if os.path.exists(VALID_IDS_FILE) else set()
    return valid_ids

def save_valid_ids(valid_ids):
    try:
        with open(VALID_IDS_FILE, 'w') as f:
            json.dump(list(valid_ids), f)
    except Exception as e:
        print(f"Error saving valid_ids: {e}")

def aggregate_time(df):
    if df.empty:
        return pd.DataFrame()
    tz = df.pivot_table(index='id', columns='zone', values='time_diff', aggfunc='sum', fill_value=0)
    for z in ('eating', 'sleeping', 'Null Zone'):
        if z not in tz.columns:
            tz[z] = 0
    return tz

def update_historical_metrics(df, current_time):
    if df.empty:
        return
    df = df.copy()
    df['hour'] = df['time'].dt.floor('h')
    for cow_id in df['id'].unique():
        if cow_id not in historical_metrics_summary:
            historical_metrics_summary[cow_id] = {}
        cow_df = df[df['id'] == cow_id]
        for hour, group in cow_df.groupby('hour'):
            if hour >= current_time:
                continue
            hour_str = hour.strftime('%Y-%m-%dT%H:%M:%SZ')
            if hour_str in historical_metrics_summary[cow_id]:
                continue
            if len(group) < 2:
                historical_metrics_summary[cow_id][hour_str] = {
                    'total_distance': 0,
                    'avg_speed': 0,
                    'time_eating_hours': 0,
                    'time_sleeping_hours': 0,
                    'time_null_zone_hours': 0,
                    'net_displacement': 0,
                    'tortuosity': None,
                    'num_stops': 0,
                    'avg_stop_duration_s': None,
                    'resting_hours': 0,
                    'zone_transitions': 0,
                    'turning_angle_variance': None,
                    'avg_nearest_neighbor_distance': None
                }
                continue
            total_dist = group['distance_meters'].sum()
            avg_speed = group['speed'].mean()
            zone_time = group.pivot_table(index='id', columns='zone', values='time_diff', aggfunc='sum', fill_value=0)
            for z in ('eating', 'sleeping', 'Null Zone'):
                if z not in zone_time.columns:
                    zone_time[z] = 0
            time_eating = zone_time.get('eating', 0).iloc[0] / 3600 if not zone_time.empty else 0
            time_sleeping = zone_time.get('sleeping', 0).iloc[0] / 3600 if not zone_time.empty else 0
            time_null = zone_time.get('Null Zone', 0).iloc[0] / 3600 if not zone_time.empty else 0
            x0, y0 = group.iloc[0][['x_smooth', 'y_smooth']]
            xN, yN = group.iloc[-1][['x_smooth', 'y_smooth']]
            net_disp = np.sqrt((xN - x0)**2 + (yN - y0)**2) / PIXELS_TO_METERS
            tortuosity = total_dist / net_disp if net_disp >= MIN_NET_DISPLACEMENT else np.nan
            stops = group['speed'] < STOP_SPEED_THRESHOLD
            n_stops = stops.astype(int).diff().abs().sum() / 2
            avg_stop_dur = group[stops]['time_diff'].mean() if stops.any() else np.nan
            resting = group[(group['speed'] < STOP_SPEED_THRESHOLD) & (group['zone'] != 'sleeping')]
            resting_hours = resting['time_diff'].sum() / 3600 if not resting.empty else 0
            zone_transitions = group['transition'].sum() if 'transition' in group else 0
            dx = group['x_smooth'].diff()
            dy = group['y_smooth'].diff()
            angles = np.arctan2(dy, dx).diff().abs().dropna()
            turning_angle_variance = angles.var() if not angles.empty else np.nan
            historical_metrics_summary[cow_id][hour_str] = {
                'total_distance': total_dist,
                'avg_speed': avg_speed,
                'time_eating_hours': time_eating,
                'time_sleeping_hours': time_sleeping,
                'time_null_zone_hours': time_null,
                'net_displacement': net_disp,
                'tortuosity': float(tortuosity) if not np.isnan(tortuosity) else None,
                'num_stops': n_stops,
                'avg_stop_duration_s': float(avg_stop_dur) if not np.isnan(avg_stop_dur) else None,
                'resting_hours': resting_hours,
                'zone_transitions': zone_transitions,
                'turning_angle_variance': float(turning_angle_variance) if not np.isnan(turning_angle_variance) else None,
                'avg_nearest_neighbor_distance': None
            }

def compute_historical_averages(cow_id, current_time):
    global last_total_distances
    hourly_metrics = historical_metrics_summary.get(cow_id, {})
    total_distance = 0
    if hourly_metrics:
        total_distance = sum(
            metrics['total_distance']
            for hour_str, metrics in hourly_metrics.items()
            if pd.to_datetime(hour_str, format='%Y-%m-%dT%H:%M:%SZ', utc=True) < current_time
        )
    if cow_id in last_total_distances:
        total_distance = max(total_distance, last_total_distances[cow_id])
    last_total_distances[cow_id] = total_distance
    if not hourly_metrics:
        return {
            'historical_avg_speed': 0,
            'historical_avg_total_distance_per_hour': 0,
            'historical_avg_time_eating_hours': 0,
            'historical_avg_time_sleeping_hours': 0,
            'historical_avg_time_null_zone_hours': 0,
            'historical_avg_net_displacement': 0,
            'historical_avg_tortuosity': np.nan,
            'historical_avg_num_stops': 0,
            'historical_avg_stop_duration_s': np.nan,
            'historical_avg_resting_hours': 0,
            'historical_avg_zone_transitions': 0,
            'historical_avg_turning_angle_variance': np.nan,
            'historical_total_distance': total_distance,
            'historical_total_num_stops': 0,
            'historical_total_zone_transitions': 0,
            'historical_total_time_eating_hours': 0,
            'historical_total_time_sleeping_hours': 0,
            'historical_total_time_null_zone_hours': 0,
            'historical_total_resting_hours': 0,
            'historical_hours_count': 0
        }
    hourly_data = [
        metrics
        for hour_str, metrics in hourly_metrics.items()
        if pd.to_datetime(hour_str, format='%Y-%m-%dT%H:%M:%SZ', utc=True) < current_time
    ]
    if not hourly_data:
        return {
            'historical_avg_speed': 0,
            'historical_avg_total_distance_per_hour': 0,
            'historical_avg_time_eating_hours': 0,
            'historical_avg_time_sleeping_hours': 0,
            'historical_avg_time_null_zone_hours': 0,
            'historical_avg_net_displacement': 0,
            'historical_avg_tortuosity': np.nan,
            'historical_avg_num_stops': 0,
            'historical_avg_stop_duration_s': np.nan,
            'historical_avg_resting_hours': 0,
            'historical_avg_zone_transitions': 0,
            'historical_avg_turning_angle_variance': np.nan,
            'historical_total_distance': total_distance,
            'historical_total_num_stops': 0,
            'historical_total_zone_transitions': 0,
            'historical_total_time_eating_hours': 0,
            'historical_total_time_sleeping_hours': 0,
            'historical_total_time_null_zone_hours': 0,
            'historical_total_resting_hours': 0,
            'historical_hours_count': 0
        }
    hourly_df = pd.DataFrame(hourly_data)
    historical_hours_count = len(hourly_df)
    total_num_stops = hourly_df['num_stops'].sum()
    total_zone_transitions = hourly_df['zone_transitions'].sum()
    total_time_eating_hours = hourly_df['time_eating_hours'].sum()
    total_time_sleeping_hours = hourly_df['time_sleeping_hours'].sum()
    total_time_null_zone_hours = hourly_df['time_null_zone_hours'].sum()
    total_resting_hours = hourly_df['resting_hours'].sum()
    return {
        'historical_avg_speed': hourly_df['avg_speed'].mean(),
        'historical_avg_total_distance_per_hour': total_distance / historical_hours_count if historical_hours_count > 0 else 0,
        'historical_avg_time_eating_hours': total_time_eating_hours / historical_hours_count if historical_hours_count > 0 else 0,
        'historical_avg_time_sleeping_hours': total_time_sleeping_hours / historical_hours_count if historical_hours_count > 0 else 0,
        'historical_avg_time_null_zone_hours': total_time_null_zone_hours / historical_hours_count if historical_hours_count > 0 else 0,
        'historical_avg_net_displacement': hourly_df['net_displacement'].mean(),
        'historical_avg_tortuosity': hourly_df['tortuosity'].mean() if 'tortuosity' in hourly_df else np.nan,
        'historical_avg_num_stops': total_num_stops / historical_hours_count if historical_hours_count > 0 else 0,
        'historical_avg_stop_duration_s': hourly_df['avg_stop_duration_s'].mean() if 'avg_stop_duration_s' in hourly_df else np.nan,
        'historical_avg_resting_hours': total_resting_hours / historical_hours_count if historical_hours_count > 0 else 0,
        'historical_avg_zone_transitions': total_zone_transitions / historical_hours_count if historical_hours_count > 0 else 0,
        'historical_avg_turning_angle_variance': hourly_df['turning_angle_variance'].mean() if 'turning_angle_variance' in hourly_df else np.nan,
        'historical_total_distance': total_distance,
        'historical_total_num_stops': total_num_stops,
        'historical_total_zone_transitions': total_zone_transitions,
        'historical_total_time_eating_hours': total_time_eating_hours,
        'historical_total_time_sleeping_hours': total_time_sleeping_hours,
        'historical_total_time_null_zone_hours': total_time_null_zone_hours,
        'historical_total_resting_hours': total_resting_hours,
        'historical_hours_count': historical_hours_count
    }

def compute_cow_metrics(cow_id, df, snapshot_time, valid_ids, downsampled_df=None):
    global last_processed_timestamps
    historical_metrics = compute_historical_averages(cow_id, snapshot_time)
    if cow_id not in df['id'].values:
        return {
            'cow_id': cow_id,
            'timestamp': snapshot_time.strftime('%Y-%m-%dT%H:%M:%SZ'),
            'hourly_total_distance': 0,
            'hourly_avg_speed': 0,
            'hourly_time_eating_hours': 0,
            'hourly_time_sleeping_hours': 0,
            'hourly_time_null_zone_hours': 0,
            'hourly_net_displacement': 0,
            'hourly_tortuosity': np.nan,
            'hourly_num_stops': 0,
            'hourly_avg_stop_duration_s': np.nan,
            'hourly_resting_hours': 0,
            'hourly_zone_transitions': 0,
            'hourly_turning_angle_variance': np.nan,
            'hourly_avg_nearest_neighbor_distance': np.nan,
            **historical_metrics
        }
    one_hour_ago = snapshot_time - pd.Timedelta(hours=1)
    subset = df[(df['id'] == cow_id) & (df['time'] >= one_hour_ago) & (df['time'] <= snapshot_time)].sort_values('time')
    print(f"Cow {cow_id}: Processing {len(subset)} records from {one_hour_ago} to {snapshot_time}")
    if len(subset) < 2:
        current_metrics = {
            'hourly_total_distance': 0,
            'hourly_avg_speed': 0,
            'hourly_time_eating_hours': 0,
            'hourly_time_sleeping_hours': 0,
            'hourly_time_null_zone_hours': 0,
            'hourly_net_displacement': 0,
            'hourly_tortuosity': np.nan,
            'hourly_num_stops': 0,
            'hourly_avg_stop_duration_s': np.nan,
            'hourly_resting_hours': 0,
            'hourly_zone_transitions': 0,
            'hourly_turning_angle_variance': np.nan,
            'hourly_avg_nearest_neighbor_distance': np.nan,
        }
    else:
        total_dist = subset['distance_meters'].sum()
        avg_speed = subset['speed'].mean()
        cow_zones = aggregate_time(subset)
        cow_zones = cow_zones.loc[cow_id] if cow_id in cow_zones.index else pd.Series({'eating': 0, 'sleeping': 0, 'Null Zone': 0})
        time_eating = cow_zones.get('eating', 0) / 3600
        time_sleeping = cow_zones.get('sleeping', 0) / 3600
        time_null = cow_zones.get('Null Zone', 0) / 3600
        x0, y0 = subset.iloc[0][['x_smooth', 'y_smooth']]
        xN, yN = subset.iloc[-1][['x_smooth', 'y_smooth']]
        net_disp = np.sqrt((xN - x0)**2 + (yN - y0)**2) / PIXELS_TO_METERS
        tortuosity = total_dist / net_disp if net_disp >= MIN_NET_DISPLACEMENT else np.nan
        stops = subset['speed'] < STOP_SPEED_THRESHOLD
        n_stops = stops.astype(int).diff().abs().sum() / 2
        avg_stop_dur = subset[stops]['time_diff'].mean() if stops.any() else np.nan
        resting = subset[(subset['speed'] < STOP_SPEED_THRESHOLD) & (subset['zone'] != 'sleeping')]
        resting_hours = resting['time_diff'].sum() / 3600 if not resting.empty else 0
        zone_transitions = subset['transition'].sum() if 'transition' in subset else 0
        dx = subset['x_smooth'].diff()
        dy = subset['y_smooth'].diff()
        angles = np.arctan2(dy, dx).diff().abs().dropna()
        turning_angle_variance = angles.var() if not angles.empty else np.nan
        avg_nearest_dist = np.nan
        if downsampled_df is not None and not downsampled_df.empty and len(downsampled_df) >= 2:
            try:
                coords = downsampled_df[['x_smooth', 'y_smooth']].values
                tree = KDTree(coords)
                cow_mask = downsampled_df['id'] == cow_id
                cow_indices = downsampled_df[cow_mask].index.tolist()
                if cow_indices:
                    distances = []
                    for idx in cow_indices:
                        if idx < len(coords):
                            dist, _ = tree.query([coords[idx]], k=2)
                            nearest_dist = dist[0][1] if len(dist[0]) > 1 else np.nan
                            nearest_dist_meters = nearest_dist / PIXELS_TO_METERS if not np.isnan(nearest_dist) else np.nan
                            distances.append(nearest_dist_meters)
                    avg_nearest_dist = np.mean(distances) if distances and not all(np.isnan(distances)) else np.nan
            except Exception as e:
                print(f"Error computing nearest neighbor for cow {cow_id}: {e}")
                avg_nearest_dist = np.nan
        current_metrics = {
            'hourly_total_distance': total_dist,
            'hourly_avg_speed': avg_speed,
            'hourly_time_eating_hours': time_eating,
            'hourly_time_sleeping_hours': time_sleeping,
            'hourly_time_null_zone_hours': time_null,
            'hourly_net_displacement': net_disp,
            'hourly_tortuosity': float(tortuosity) if not np.isnan(tortuosity) else None,
            'hourly_num_stops': n_stops,
            'hourly_avg_stop_duration_s': float(avg_stop_dur) if not np.isnan(avg_stop_dur) else None,
            'hourly_resting_hours': resting_hours,
            'hourly_zone_transitions': zone_transitions,
            'hourly_turning_angle_variance': float(turning_angle_variance) if not np.isnan(turning_angle_variance) else None,
            'hourly_avg_nearest_neighbor_distance': float(avg_nearest_dist) if not np.isnan(avg_nearest_dist) else None,
        }
    if not subset.empty:
        last_processed_timestamps[cow_id] = subset['time'].max()
    return {
        'cow_id': cow_id,
        'timestamp': snapshot_time.strftime('%Y-%m-%dT%H:%M:%SZ'),
        **current_metrics,
        **historical_metrics
    }

def compute_activity_snapshot(current_df, timestamp, valid_ids):
    global latest_snapshot
    if current_df.empty:
        print(f"No data to compute activity snapshot at {timestamp}")
        return pd.DataFrame()
    snapshot_time = pd.to_datetime(timestamp, format='%Y-%m-%dT%H:%M:%SZ', utc=True)
    print(f"\n=== Snapshot Update at {timestamp} ===")
    print(f"Current data rows: {len(current_df)}")
    print(f"Valid cow IDs: {sorted(valid_ids)}")
    one_hour_ago = snapshot_time - pd.Timedelta(hours=1)
    hourly_subset = current_df[(current_df['time'] >= one_hour_ago) & (current_df['time'] <= snapshot_time)].copy()
    downsampled_df = None
    if not hourly_subset.empty:
        print(f"Downsampling data for nearest neighbor calculation: {len(hourly_subset)} rows")
        downsampled_df = (hourly_subset.set_index('time')
                         .groupby('id', group_keys=True)
                         .resample('10s')
                         .last()
                         .reset_index(level=0, drop=True)
                         .reset_index())
        if not downsampled_df.empty:
            downsampled_df = downsampled_df.dropna(subset=['x_smooth', 'y_smooth']).reset_index(drop=True)
        print(f"After downsampling to 10s intervals: {len(downsampled_df)} rows")
    try:
        with Parallel(n_jobs=multiprocessing.cpu_count(), backend='threading') as parallel:
            results = parallel(
                delayed(compute_cow_metrics)(
                    cow_id,
                    current_df,
                    snapshot_time,
                    valid_ids,
                    downsampled_df
                )
                for cow_id in valid_ids
            )
        if not results:
            print("No valid results after computation")
            return pd.DataFrame()
        snapshot_df = pd.DataFrame(results)
        if not snapshot_df.empty:
            snapshot_df.sort_values(['timestamp', 'cow_id'], inplace=True)
            print(f"Computed activity snapshot with {len(snapshot_df)} rows")
        with data_lock:
            latest_snapshot = snapshot_df
            two_hours_ago = snapshot_time - pd.Timedelta(hours=2)
            globals()['current_df'] = current_df[current_df['time'] >= two_hours_ago]
            print(f"Cleaned current_df to {len(globals()['current_df'])} rows (data since {two_hours_ago})")
        return snapshot_df
    except Exception as e:
        print(f"Error during snapshot computation at {timestamp}: {e}")
        return pd.DataFrame()

def upload_activity_snapshot(snapshot_df, timestamp):
    if snapshot_df.empty:
        print(f"No data to upload for activity snapshot at {timestamp}")
        return
    batch = db.batch()
    for idx, row in snapshot_df.iterrows():
        ref = db.collection('activity_snapshots').document(timestamp).collection('snapshots').document(row['cow_id'])
        batch.set(ref, row.to_dict())
        if (idx + 1) % BATCH_SIZE == 0:
            try:
                batch.commit()
            except Exception as e:
                print(f"Error committing activity snapshot batch: {e}")
            batch = db.batch()
    try:
        batch.commit()
        print(f"Uploaded activity snapshot with {len(snapshot_df)} records")
        upload_plots_to_storage(timestamp)
    except Exception as e:
        print(f"Error uploading activity snapshot to Firestore: {e}")


def upload_plots_to_storage(timestamp):
    if not os.path.exists(OUTPUT_DIR):
        print(f"No output directory {OUTPUT_DIR} found for plots upload")
        return

    plot_files = [f for f in os.listdir(OUTPUT_DIR) if f.endswith('.png')]
    if not plot_files:
        print(f"No plots found in {OUTPUT_DIR} for upload")
        return

    def _upload_one(plot_filename):
        local_path = os.path.join(OUTPUT_DIR, plot_filename)
        storage_path = f'plots/{timestamp}/{plot_filename}'
        blob = bucket.blob(storage_path)
        blob.upload_from_filename(local_path)
        return plot_filename, storage_path

    # choose a pool size that makes sense for your environment
    with ThreadPoolExecutor(max_workers=8) as executor:
        futures = {executor.submit(_upload_one, fn): fn for fn in plot_files}
        for future in as_completed(futures):
            fn = futures[future]
            try:
                plot_filename, storage_path = future.result()
                print(f"Uploaded {plot_filename} → {storage_path}")
            except Exception as e:
                print(f"Error uploading {fn}: {e}")


def print_analysis_summary(current_df, snapshot_df, timestamp):
    if current_df.empty:
        print(f"No data to summarize at {timestamp}")
        return
    print("\n=== Analysis Summary ===")
    print(f"Timestamp: {timestamp}")
    print(f"Number of records processed: {len(current_df)}")
    print(f"Number of cows: {len(snapshot_df)}")
    print("\n=== Hourly Metrics Summary ===")
    summary_cols = ['cow_id', 'hourly_total_distance', 'hourly_avg_speed', 'hourly_time_eating_hours',
                    'hourly_time_sleeping_hours', 'hourly_time_null_zone_hours', 'hourly_num_stops',
                    'hourly_avg_stop_duration_s', 'hourly_zone_transitions', 'hourly_avg_nearest_neighbor_distance']
    if not snapshot_df.empty:
        print(snapshot_df[summary_cols].to_string(index=False))
    print("\n=== Activity Snapshot Data ===")
    if snapshot_df.empty:
        print("No snapshot data")
    else:
        for _, row in snapshot_df.iterrows():
            print(f"\nCow ID: {row['cow_id']}")
            print("Hourly Metrics:")
            print(f"  Total Distance: {row['hourly_total_distance']:.2f} meters")
            print(f"  Average Speed: {row['hourly_avg_speed']:.2f} m/s")
            print(f"  Time in Eating Zone: {row['hourly_time_eating_hours']:.2f} hours")
            print(f"  Time in Sleeping Zone: {row['hourly_time_sleeping_hours']:.2f} hours")
            print(f"  Time in Null Zone: {row['hourly_time_null_zone_hours']:.2f} hours")
            print(f"  Net Displacement: {row['hourly_net_displacement']:.2f} meters")
            print(f"  Tortuosity: {row['hourly_tortuosity']:.2f}" if not pd.isna(row['hourly_tortuosity']) else "  Tortuosity: N/A")
            print(f"  Number of Stops: {row['hourly_num_stops']}")
            print(f"  Average Stop Duration: {row['hourly_avg_stop_duration_s']:.2f} seconds" if not pd.isna(row['hourly_avg_stop_duration_s']) else "  Average Stop Duration: N/A")
            print(f"  Resting Hours: {row['hourly_resting_hours']:.2f} hours")
            print(f"  Zone Transitions: {row['hourly_zone_transitions']}")
            print(f"  Turning Angle Variance: {row['hourly_turning_angle_variance']:.2f}" if not pd.isna(row['hourly_turning_angle_variance']) else "  Turning Angle Variance: N/A")
            print(f"  Average Nearest Neighbor Distance: {row['hourly_avg_nearest_neighbor_distance']:.2f} meters" if not pd.isna(row['hourly_avg_nearest_neighbor_distance']) else "  Average Nearest Neighbor Distance: N/A")
            print("Historical Metrics:")
            print(f"  Historical Average Speed: {row['historical_avg_speed']:.2f} m/s")
            print(f"  Historical Average Total Distance per Hour: {row['historical_avg_total_distance_per_hour']:.2f} meters/hour")
            print(f"  Historical Average Time in Eating Zone: {row['historical_avg_time_eating_hours']:.2f} hours")
            print(f"  Historical Average Time in Sleeping Zone: {row['historical_avg_time_sleeping_hours']:.2f} hours")
            print(f"  Historical Average Time in Null Zone: {row['historical_avg_time_null_zone_hours']:.2f} hours")
            print(f"  Historical Average Net Displacement: {row['historical_avg_net_displacement']:.2f} meters")
            print(f"  Historical Average Tortuosity: {row['historical_avg_tortuosity']:.2f}" if not pd.isna(row['historical_avg_tortuosity']) else "  Historical Average Tortuosity: N/A")
            print(f"  Historical Average Number of Stops: {row['historical_avg_num_stops']:.2f}")
            print(f"  Historical Average Stop Duration: {row['historical_avg_stop_duration_s']:.2f} seconds" if not pd.isna(row['historical_avg_stop_duration_s']) else "  Historical Average Stop Duration: N/A")
            print(f"  Historical Average Resting Hours: {row['historical_avg_resting_hours']:.2f} hours")
            print(f"  Historical Average Zone Transitions: {row['historical_avg_zone_transitions']:.2f}")
            print(f"  Historical Average Turning Angle Variance: {row['historical_avg_turning_angle_variance']:.2f}" if not pd.isna(row['historical_avg_turning_angle_variance']) else "  Historical Average Turning Angle Variance: N/A")
            print(f"  Historical Total Distance: {row['historical_total_distance']:.2f} meters")
            print(f"  Historical Total Number of Stops: {row['historical_total_num_stops']}")
            print(f"  Historical Total Zone Transitions: {row['historical_total_zone_transitions']}")
            print(f"  Historical Total Time in Eating Zone: {row['historical_total_time_eating_hours']:.2f} hours")
            print(f"  Historical Total Time in Sleeping Zone: {row['historical_total_time_sleeping_hours']:.2f} hours")
            print(f"  Historical Total Time in Null Zone: {row['historical_total_time_null_zone_hours']:.2f} hours")
            print(f"  Historical Total Resting Hours: {row['historical_total_resting_hours']:.2f} hours")
            print(f"  Historical Hours Count: {row['historical_hours_count']}")

def get_historical_metrics_df():
    historical_metrics_list = []
    for cow_id, hours in historical_metrics_summary.items():
        for hour_str, metrics in hours.items():
            metrics['cow_id'] = cow_id
            metrics['hour'] = hour_str
            historical_metrics_list.append(metrics)
    if not historical_metrics_list:
        return pd.DataFrame()
    historical_metrics_df = pd.DataFrame(historical_metrics_list)
    historical_metrics_df['hour'] = pd.to_datetime(historical_metrics_df['hour'], format='%Y-%m-%dT%H:%M:%SZ', utc=True)
    historical_metrics_df['date'] = historical_metrics_df['hour'].dt.date
    return historical_metrics_df

def spatial_heatmap(df, timestamp, by_day=False):
    if df.empty:
        return
    dfp = df.copy()
    if by_day:
        dfp['date'] = dfp['time'].dt.date
        for d in sorted(dfp['date'].unique()):
            sub = dfp[dfp['date'] == d]
            if sub.empty:
                continue
            plt.figure(figsize=(10, 6))
            plt.hexbin(sub['x'] / PIXELS_TO_METERS, sub['y'] / PIXELS_TO_METERS, gridsize=40, cmap='hot', mincnt=1, alpha=0.7)
            plt.colorbar(label='Number of Observations')
            plt.title(f"Spatial Heatmap - All Cows (Day {d})")
            plt.xlabel('X Position (meters)')
            plt.ylabel('Y Position (meters)')
            plt.gca().invert_yaxis()
            plt.grid(True, linestyle='--', alpha=0.5)
            plt.savefig(f"{OUTPUT_DIR}/heatmap_day_{d}_{timestamp}.png")
            plt.close()
            for cow_id in sub['id'].unique():
                cow_sub = sub[sub['id'] == cow_id]
                if cow_sub.empty:
                    continue
                plt.figure(figsize=(10, 6))
                plt.hexbin(cow_sub['x'] / PIXELS_TO_METERS, cow_sub['y'] / PIXELS_TO_METERS, gridsize=40, cmap='hot', mincnt=1, alpha=0.7)
                plt.colorbar(label='Number of Observations')
                plt.title(f"Spatial Heatmap (Cow {cow_id} - Day {d})")
                plt.xlabel('X Position (meters)')
                plt.ylabel('Y Position (meters)')
                plt.gca().invert_yaxis()
                plt.grid(True, linestyle='--', alpha=0.5)
                plt.savefig(f"{OUTPUT_DIR}/heatmap_cow_{cow_id}_day_{d}_{timestamp}.png")
                plt.close()
    else:
        plt.figure(figsize=(10, 6))
        plt.hexbin(dfp['x'] / PIXELS_TO_METERS, dfp['y'] / PIXELS_TO_METERS, gridsize=40, cmap='hot', mincnt=1, alpha=0.7)
        plt.colorbar(label='Number of Observations')
        plt.title("Spatial Heatmap - All Cows (Overall)")
        plt.xlabel('X Position (meters)')
        plt.ylabel('Y Position (meters)')
        plt.gca().invert_yaxis()
        plt.grid(True, linestyle='--', alpha=0.5)
        plt.savefig(f"{OUTPUT_DIR}/heatmap_overall_{timestamp}.png")
        plt.close()
        for cow_id in dfp['id'].unique():
            cow_sub = dfp[dfp['id'] == cow_id]
            if cow_sub.empty:
                continue
            plt.figure(figsize=(10, 6))
            plt.hexbin(cow_sub['x'] / PIXELS_TO_METERS, cow_sub['y'] / PIXELS_TO_METERS, gridsize=40, cmap='hot', mincnt=1, alpha=0.7)
            plt.colorbar(label='Number of Observations')
            plt.title(f"Spatial Heatmap (Cow {cow_id} - Overall)")
            plt.xlabel('X Position (meters)')
            plt.ylabel('Y Position (meters)')
            plt.gca().invert_yaxis()
            plt.grid(True, linestyle='--', alpha=0.5)
            plt.savefig(f"{OUTPUT_DIR}/heatmap_cow_{cow_id}_overall_{timestamp}.png")
            plt.close()

def zone_transition_matrix(df, timestamp, by_day=False):
    if df.empty:
        return
    # make a full copy first
    dft = df.copy()
    dft['prev_zone'] = dft.groupby('id')['zone'].shift(1)
    # dropna *and* call .copy() on the result so that dft is always a true independent DataFrame
    dft = dft.dropna(subset=['prev_zone']).copy()

    if by_day:
        # now it's safe to assign new columns without a SettingWithCopyWarning
        dft['date'] = dft['time'].dt.date
        for d in sorted(dft['date'].unique()):
            sub = dft[dft['date'] == d]
            if sub.empty:
                continue
            mat = pd.crosstab(sub['prev_zone'], sub['zone'])
            plt.figure(figsize=(8, 6))
            sns.heatmap(mat, annot=True, fmt='d', cmap='YlGnBu', cbar_kws={'label': 'Number of Transitions'})
            plt.title(f"Zone Transition Matrix - All Cows (Day {d})")
            plt.xlabel('To Zone')
            plt.ylabel('From Zone')
            plt.savefig(f"{OUTPUT_DIR}/trans_matrix_day_{d}_{timestamp}.png")
            plt.close()
    else:
        mat = pd.crosstab(dft['prev_zone'], dft['zone'])
        plt.figure(figsize=(8, 6))
        sns.heatmap(mat, annot=True, fmt='d', cmap='YlGnBu', cbar_kws={'label': 'Number of Transitions'})
        plt.title("Zone Transition Matrix - All Cows (Overall)")
        plt.xlabel('To Zone')
        plt.ylabel('From Zone')
        plt.savefig(f"{OUTPUT_DIR}/trans_matrix_overall_{timestamp}.png")
        plt.close()


def temporal_activity_profiles(df, timestamp, by_day=False):
    if df.empty:
        return
    dfp = df.copy()
    if by_day:
        dfp['date'] = dfp['time'].dt.date
        for d in sorted(dfp['date'].unique()):
            sub = dfp[dfp['date'] == d]
            if sub.empty:
                continue
            sub['hour'] = sub['time'].dt.hour + sub['time'].dt.minute / 60
            hd = sub.groupby('hour')['distance_meters'].mean()
            hs = sub.groupby('hour')['speed'].mean()
            fig, ax1 = plt.subplots(figsize=(10, 6))
            ax1.bar(hd.index, hd, color='skyblue', alpha=0.7, label='Avg Distance')
            ax1.set_xlabel('Hour of Day')
            ax1.set_ylabel('Average Distance (meters)')
            ax2 = ax1.twinx()
            ax2.plot(hs.index, hs, marker='o', color='salmon', label='Avg Speed')
            ax2.set_ylabel('Average Speed (m/s)')
            fig.legend(loc='upper left', bbox_to_anchor=(0.1, 0.9))
            plt.title(f"Temporal Activity Profiles - All Cows (Day {d})")
            plt.grid(True, linestyle='--', alpha=0.5)
            plt.savefig(f"{OUTPUT_DIR}/temp_activity_day_{d}_{timestamp}.png")
            plt.close()
            for cow_id in sub['id'].unique():
                cow_sub = sub[sub['id'] == cow_id]
                if cow_sub.empty:
                    continue
                hd = cow_sub.groupby('hour')['distance_meters'].mean()
                hs = cow_sub.groupby('hour')['speed'].mean()
                fig, ax1 = plt.subplots(figsize=(10, 6))
                ax1.bar(hd.index, hd, color='skyblue', alpha=0.7, label='Avg Distance')
                ax1.set_xlabel('Hour of Day')
                ax1.set_ylabel('Average Distance (meters)')
                ax2 = ax1.twinx()
                ax2.plot(hs.index, hs, marker='o', color='salmon', label='Avg Speed')
                ax2.set_ylabel('Average Speed (m/s)')
                fig.legend(loc='upper left', bbox_to_anchor=(0.1, 0.9))
                plt.title(f"Temporal Activity Profiles (Cow {cow_id} - Day {d})")
                plt.grid(True, linestyle='--', alpha=0.5)
                plt.savefig(f"{OUTPUT_DIR}/temp_activity_cow_{cow_id}_day_{d}_{timestamp}.png")
                plt.close()
    else:
        dfp['hour'] = dfp['time'].dt.hour + dfp['time'].dt.minute / 60
        hd = dfp.groupby('hour')['distance_meters'].mean()
        hs = dfp.groupby('hour')['speed'].mean()
        fig, ax1 = plt.subplots(figsize=(10, 6))
        ax1.bar(hd.index, hd, color='skyblue', alpha=0.7, label='Avg Distance')
        ax1.set_xlabel('Hour of Day')
        ax1.set_ylabel('Average Distance (meters)')
        ax2 = ax1.twinx()
        ax2.plot(hs.index, hs, marker='o', color='salmon', label='Avg Speed')
        ax2.set_ylabel('Average Speed (m/s)')
        fig.legend(loc='upper left', bbox_to_anchor=(0.1, 0.9))
        plt.title("Temporal Activity Profiles - All Cows (Overall)")
        plt.grid(True, linestyle='--', alpha=0.5)
        plt.savefig(f"{OUTPUT_DIR}/temp_activity_overall_{timestamp}.png")
        plt.close()
        for cow_id in dfp['id'].unique():
            cow_sub = dfp[dfp['id'] == cow_id]
            if cow_sub.empty:
                continue
            hd = cow_sub.groupby('hour')['distance_meters'].mean()
            hs = cow_sub.groupby('hour')['speed'].mean()
            fig, ax1 = plt.subplots(figsize=(10, 6))
            ax1.bar(hd.index, hd, color='skyblue', alpha=0.7, label='Avg Distance')
            ax1.set_xlabel('Hour of Day')
            ax1.set_ylabel('Average Distance (meters)')
            ax2 = ax1.twinx()
            ax2.plot(hs.index, hs, marker='o', color='salmon', label='Avg Speed')
            ax2.set_ylabel('Average Speed (m/s)')
            fig.legend(loc='upper left', bbox_to_anchor=(0.1, 0.9))
            plt.title(f"Temporal Activity Profiles (Cow {cow_id} - Overall)")
            plt.grid(True, linestyle='--', alpha=0.5)
            plt.savefig(f"{OUTPUT_DIR}/temp_activity_cow_{cow_id}_overall_{timestamp}.png")
            plt.close()

def behavioral_clustering(df, timestamp, by_day=False):
    if df.empty:
        return
    dfc = df.copy()
    if by_day:
        dfc['date'] = dfc['time'].dt.date
        for d in sorted(dfc['date'].unique()):
            sub = dfc[dfc['date'] == d]
            if sub.empty:
                continue
            feats = sub.groupby('id').agg(
                total_distance=('distance_meters', 'sum'),
                avg_speed=('speed', 'mean'),
                total_time=('time_diff', 'sum')
            )
            zt = aggregate_time(sub)
            feats = feats.join(zt, how='left').fillna(0)
            sc = StandardScaler().fit_transform(feats)
            labels = KMeans(n_clusters=3, random_state=42).fit_predict(sc)
            feats['cluster'] = labels
            plt.figure(figsize=(10, 6))
            sns.scatterplot(data=feats, x='total_distance', y='avg_speed', hue='cluster', palette='colorblind', s=100, alpha=0.7)
            plt.title(f"Behavioral Clustering - All Cows (Day {d})")
            plt.xlabel('Total Distance (meters)')
            plt.ylabel('Average Speed (m/s)')
            plt.legend(title='Cluster')
            plt.grid(True, linestyle='--', alpha=0.5)
            plt.savefig(f"{OUTPUT_DIR}/clustering_day_{d}_{timestamp}.png")
            plt.close()
    else:
        feats = dfc.groupby('id').agg(
            total_distance=('distance_meters', 'sum'),
            avg_speed=('speed', 'mean'),
            total_time=('time_diff', 'sum')
        )
        zt = aggregate_time(dfc)
        feats = feats.join(zt, how='left').fillna(0)
        sc = StandardScaler().fit_transform(feats)
        labels = KMeans(n_clusters=3, random_state=42).fit_predict(sc)
        feats['cluster'] = labels
        plt.figure(figsize=(10, 6))
        sns.scatterplot(data=feats, x='total_distance', y='avg_speed', hue='cluster', palette='colorblind', s=100, alpha=0.7)
        plt.title("Behavioral Clustering - All Cows (Overall)")
        plt.xlabel('Total Distance (meters)')
        plt.ylabel('Average Speed (m/s)')
        plt.legend(title='Cluster')
        plt.grid(True, linestyle='--', alpha=0.5)
        plt.savefig(f"{OUTPUT_DIR}/clustering_overall_{timestamp}.png")
        plt.close()

def trajectory_analysis(df, timestamp, by_day=False):
    if df.empty:
        return
    def turns(sub):
        dx = sub['x_smooth'].diff()
        dy = sub['y_smooth'].diff()
        ang = np.arctan2(dy, dx)
        return ang.diff().abs().dropna()
    dft = df.copy()
    if by_day:
        dft['date'] = dft['time'].dt.date
        for d in sorted(dft['date'].unique()):
            sub = dft[dft['date'] == d]
            if sub.empty:
                continue
            angles = []
            for cid in sub['id'].unique():
                angles.extend(turns(sub[sub['id'] == cid]).tolist())
            if not angles:
                continue
            plt.figure(figsize=(10, 6))
            plt.hist(angles, bins=30, color='teal', alpha=0.7)
            plt.title(f"Turning Angle Distribution - All Cows (Day {d})")
            plt.xlabel('Turning Angle (radians)')
            plt.ylabel('Frequency')
            plt.grid(True, linestyle='--', alpha=0.5)
            plt.savefig(f"{OUTPUT_DIR}/trajectory_day_{d}_{timestamp}.png")
            plt.close()
    else:
        angles = []
        for cid in dft['id'].unique():
            angles.extend(turns(dft[dft['id'] == cid]).tolist())
        if not angles:
            return
        plt.figure(figsize=(10, 6))
        plt.hist(angles, bins=30, color='teal', alpha=0.7)
        plt.title("Turning Angle Distribution - All Cows (Overall)")
        plt.xlabel('Turning Angle (radians)')
        plt.ylabel('Frequency')
        plt.grid(True, linestyle='--', alpha=0.5)
        plt.savefig(f"{OUTPUT_DIR}/trajectory_overall_{timestamp}.png")
        plt.close()

def per_cow_detailed_time_series(df, timestamp, by_day=False):
    if df.empty:
        return
    df_sorted = df.sort_values('time').copy()
    if by_day:
        df_sorted['date'] = df_sorted['time'].dt.date
        for day in sorted(df_sorted['date'].unique()):
            day_df = df_sorted[df_sorted['date'] == day]
            if len(day_df) < 2:
                continue
            for cow_id, group in day_df.groupby('id'):
                if len(group) < 2:
                    continue
                group = group.drop_duplicates(subset=['time'])
                if len(group) < 2:
                    continue
                cow_data = group.set_index('time').resample('1min').ffill()
                rolling_dist = cow_data['distance_meters'].rolling('60min', min_periods=1).sum()
                rolling_speed = cow_data['speed'].rolling('60min', min_periods=1).mean()
                plt.figure(figsize=(12, 6))
                plt.plot(rolling_dist.index, rolling_dist.values, color='green', label='Rolling Distance')
                plt.xlabel('Time')
                plt.ylabel('Distance (meters)')
                ax2 = plt.twinx()
                ax2.plot(rolling_speed.index, rolling_speed.values, color='orange', label='Rolling Speed')
                ax2.set_ylabel('Speed (m/s)')
                plt.title(f"Time Series (Cow {cow_id} - Day {day})")
                plt.legend(loc='upper left')
                ax2.legend(loc='upper right')
                plt.grid(True, linestyle='--', alpha=0.5)
                plt.xticks(rotation=45)
                plt.tight_layout()
                plt.savefig(f"{OUTPUT_DIR}/timeseries_cow_{cow_id}_day_{day}_{timestamp}.png")
                plt.close()
    else:
        for cow_id, group in df_sorted.groupby('id'):
            if len(group) < 2:
                continue
            group = group.drop_duplicates(subset=['time'])
            if len(group) < 2:
                continue
            cow_data = group.set_index('time').resample('1min').ffill()
            rolling_dist = cow_data['distance_meters'].rolling('60min', min_periods=1).sum()
            rolling_speed = cow_data['speed'].rolling('60min', min_periods=1).mean()
            plt.figure(figsize=(12, 6))
            plt.plot(rolling_dist.index, rolling_dist.values, color='green', label='Rolling Distance')
            plt.xlabel('Time')
            plt.ylabel('Distance (meters)')
            ax2 = plt.twinx()
            ax2.plot(rolling_speed.index, rolling_speed.values, color='orange', label='Rolling Speed')
            ax2.set_ylabel('Speed (m/s)')
            plt.title(f"Time Series (Cow {cow_id} - Overall)")
            plt.legend(loc='upper left')
            ax2.legend(loc='upper right')
            plt.grid(True, linestyle='--', alpha=0.5)
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.savefig(f"{OUTPUT_DIR}/timeseries_cow_{cow_id}_overall_{timestamp}.png")
            plt.close()

def per_cow_zone_usage(df, timestamp):
    if df.empty:
        return
    df_copy = df.copy()
    df_copy['date'] = df_copy['time'].dt.date
    usage = df_copy.groupby(['id', 'date', 'zone'])['time_diff'].sum().reset_index()
    usage['hours'] = usage['time_diff'] / 3600
    for cow_id in usage['id'].unique():
        cow_usage = usage[usage['id'] == cow_id]
        pivoted = cow_usage.pivot(index='date', columns='zone', values='hours').fillna(0)
        cols = [c for c in ['eating', 'sleeping', 'Null Zone'] if c in pivoted]
        if not cols:
            continue
        pivoted[cols].plot(kind='bar', stacked=True, figsize=(12, 6), color=sns.color_palette('colorblind', len(cols)))
        plt.title(f"Zone Usage by Day (Cow {cow_id})")
        plt.xlabel('Date')
        plt.ylabel('Time (hours)')
        plt.legend(labels=cols, title='Zone')
        plt.grid(True, linestyle='--', alpha=0.5)
        plt.tight_layout()
        plt.savefig(f"{OUTPUT_DIR}/zone_usage_cow_{cow_id}_{timestamp}.png")
        plt.close()

def per_cow_resting_analysis(df, zone_time, timestamp, by_day=False):
    if df.empty:
        return
    df_copy = df.copy()
    df_copy['date'] = df_copy['time'].dt.date
    resting = df_copy[(df_copy['speed'] < STOP_SPEED_THRESHOLD) & (df_copy['zone'] != 'sleeping')]
    resting_hours = resting.groupby(['id', 'date'])['time_diff'].sum().reset_index()
    resting_hours['hours'] = resting_hours['time_diff'] / 3600
    sleeping_hours = zone_time.get('sleeping', pd.Series(0)).reset_index()
    sleeping_hours = sleeping_hours.rename(columns={'index': 'id', 'sleeping': 'hours'})
    sleeping_hours['date'] = df_copy['date'].max() if by_day else None
    sleeping_hours['type'] = 'Sleeping'
    resting_hours = resting_hours[['id', 'date', 'hours']].copy()
    resting_hours['type'] = 'Resting (Non-Sleeping)'
    combined = pd.concat([resting_hours, sleeping_hours], ignore_index=True)
    if by_day:
        for cow_id in combined['id'].unique():
            cow_data = combined[combined['id'] == cow_id]
            if cow_data.empty:
                continue
            pivoted = cow_data.pivot(index='date', columns='type', values='hours').fillna(0)
            pivoted.plot(kind='bar', stacked=True, figsize=(12, 6), color=['forestgreen', 'purple'])
            plt.title(f"Resting and Sleeping Hours by Day (Cow {cow_id})")
            plt.xlabel('Date')
            plt.ylabel('Time (hours)')
            plt.legend(title='Activity Type')
            plt.grid(True, linestyle='--', alpha=0.5)
            plt.tight_layout()
            plt.savefig(f"{OUTPUT_DIR}/resting_cow_{cow_id}_day_{timestamp}.png")
            plt.close()
    else:
        for cow_id in combined['id'].unique():
            cow_data = combined[combined['id'] == cow_id]
            if cow_data.empty:
                continue
            pivoted = cow_data.groupby('type')['hours'].sum().reset_index()
            pivoted.plot(kind='bar', x='type', y='hours', figsize=(10, 6), color=['forestgreen', 'purple'], legend=False)
            plt.title(f"Resting and Sleeping Hours (Cow {cow_id} - Overall)")
            plt.xlabel('Activity Type')
            plt.ylabel('Total Time (hours)')
            plt.grid(True, linestyle='--', alpha=0.5)
            plt.tight_layout()
            plt.savefig(f"{OUTPUT_DIR}/resting_cow_{cow_id}_overall_{timestamp}.png")
            plt.close()

def per_cow_turning_analysis(df, timestamp, by_day=False):
    if df.empty:
        return
    df_copy = df.copy()
    df_copy['date'] = df_copy['time'].dt.date
    results = []
    for cow_id in df_copy['id'].unique():
        subset = df_copy[df_copy['id'] == cow_id]
        if len(subset) < 2:
            continue
        if by_day:
            for day in sorted(subset['date'].unique()):
                day_subset = subset[subset['date'] == day]
                if len(day_subset) < 2:
                    continue
                dx = day_subset['x_smooth'].diff()
                dy = day_subset['y_smooth'].diff()
                angles = np.arctan2(dy, dx).diff().abs().dropna()
                variance = angles.var() if not angles.empty else np.nan
                results.append({'id': cow_id, 'date': day, 'turning_angle_variance': variance})
        else:
            dx = subset['x_smooth'].diff()
            dy = subset['y_smooth'].diff()
            angles = np.arctan2(dy, dx).diff().abs().dropna()
            variance = angles.var() if not angles.empty else np.nan
            results.append({'id': cow_id, 'date': None, 'turning_angle_variance': variance})
    turning_df = pd.DataFrame(results)
    if turning_df.empty:
        return
    if by_day:
        for cow_id in turning_df['id'].unique():
            cow_data = turning_df[turning_df['id'] == cow_id]
            if cow_data.empty or cow_data['turning_angle_variance'].isna().all():
                continue
            plt.figure(figsize=(12, 6))
            plt.bar(cow_data['date'].astype(str), cow_data['turning_angle_variance'], color='teal')
            plt.title(f"Turning Angle Variance by Day (Cow {cow_id})")
            plt.xlabel('Date')
            plt.ylabel('Variance (radians²)')
            plt.grid(True, linestyle='--', alpha=0.5)
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.savefig(f"{OUTPUT_DIR}/turning_cow_{cow_id}_day_{timestamp}.png")
            plt.close()
    else:
        for cow_id in turning_df['id'].unique():
            cow_data = turning_df[turning_df['id'] == cow_id]
            if cow_data.empty or pd.isna(cow_data['turning_angle_variance'].iloc[0]):
                continue
            plt.figure(figsize=(10, 6))
            plt.bar([cow_id], cow_data['turning_angle_variance'], color='teal')
            plt.title(f"Turning Angle Variance (Cow {cow_id} - Overall)")
            plt.xlabel('Cow ID')
            plt.ylabel('Variance (radians²)')
            plt.grid(True, linestyle='--', alpha=0.5)
            plt.tight_layout()
            plt.savefig(f"{OUTPUT_DIR}/turning_cow_{cow_id}_overall_{timestamp}.png")
            plt.close()

def per_cow_path_trajectory(df, timestamp, by_day=False):
    if df.empty:
        return
    df_copy = df.copy()
    if by_day:
        df_copy['date'] = df_copy['time'].dt.date
        for day in sorted(df_copy['date'].unique()):
            day_df = df_copy[df_copy['date'] == day]
            for cow_id in day_df['id'].unique():
                subset = day_df[day_df['id'] == cow_id]
                if len(subset) < 2:
                    continue
                plt.figure(figsize=(10, 6))
                plt.scatter(subset['x_smooth'] / PIXELS_TO_METERS, subset['y_smooth'] / PIXELS_TO_METERS, c=subset['speed'], cmap='viridis', s=20, alpha=0.6)
                plt.plot(subset['x_smooth'] / PIXELS_TO_METERS, subset['y_smooth'] / PIXELS_TO_METERS, color='gray', linewidth=1, alpha=0.5)
                plt.colorbar(label='Speed (m/s)')
                plt.gca().invert_yaxis()
                plt.title(f"Path Trajectory (Cow {cow_id} - Day {day})")
                plt.xlabel('X Position (meters)')
                plt.ylabel('Y Position (meters)')
                plt.grid(True, linestyle='--', alpha=0.5)
                plt.savefig(f"{OUTPUT_DIR}/trajectory_cow_{cow_id}_day_{day}_{timestamp}.png")
                plt.close()
    else:
        for cow_id in df_copy['id'].unique():
            subset = df_copy[df_copy['id'] == cow_id]
            if len(subset) < 2:
                continue
            plt.figure(figsize=(10, 6))
            plt.scatter(subset['x_smooth'] / PIXELS_TO_METERS, subset['y_smooth'] / PIXELS_TO_METERS, c=subset['speed'], cmap='viridis', s=20, alpha=0.6)
            plt.plot(subset['x_smooth'] / PIXELS_TO_METERS, subset['y_smooth'] / PIXELS_TO_METERS, color='gray', linewidth=1, alpha=0.5)
            plt.colorbar(label='Speed (m/s)')
            plt.gca().invert_yaxis()
            plt.title(f"Path Trajectory (Cow {cow_id} - Overall)")
            plt.xlabel('X Position (meters)')
            plt.ylabel('Y Position (meters)')
            plt.grid(True, linestyle='--', alpha=0.5)
            plt.savefig(f"{OUTPUT_DIR}/trajectory_cow_{cow_id}_overall_{timestamp}.png")
            plt.close()

def per_cow_advanced_metrics(df, timestamp):
    if df.empty:
        return
    metrics_df = get_historical_metrics_df()
    if metrics_df.empty:
        print("\nNo historical metrics available for advanced metrics plotting")
        return
    for cow_id in metrics_df['cow_id'].unique():
        cow_data = metrics_df[metrics_df['cow_id'] == cow_id]
        if cow_data.empty:
            continue
        for date in sorted(cow_data['date'].unique()):
            day_data = cow_data[cow_data['date'] == date]
            if day_data.empty:
                continue
            if len(day_data['hour']) == len(day_data['total_distance']) and not day_data['total_distance'].isna().all():
                plt.figure(figsize=(12, 6))
                plt.plot(day_data['hour'], day_data['total_distance'], marker='o', color='blue')
                plt.title(f"Total Distance (Cow {cow_id} - Day {date})")
                plt.xlabel('Time')
                plt.ylabel('Distance (meters)')
                plt.grid(True, linestyle='--', alpha=0.5)
                plt.xticks(rotation=45)
                plt.tight_layout()
                plt.savefig(f"{OUTPUT_DIR}/total_distance_cow_{cow_id}_day_{date}_{timestamp}.png")
                plt.close()
            if len(day_data['hour']) == len(day_data['avg_speed']) and not day_data['avg_speed'].isna().all():
                plt.figure(figsize=(12, 6))
                plt.plot(day_data['hour'], day_data['avg_speed'], marker='o', color='orange')
                plt.title(f"Average Speed (Cow {cow_id} - Day {date})")
                plt.xlabel('Time')
                plt.ylabel('Speed (m/s)')
                plt.grid(True, linestyle='--', alpha=0.5)
                plt.xticks(rotation=45)
                plt.tight_layout()
                plt.savefig(f"{OUTPUT_DIR}/avg_speed_cow_{cow_id}_day_{date}_{timestamp}.png")
                plt.close()
            zones = ['time_eating_hours', 'time_sleeping_hours', 'time_null_zone_hours']
            zone_data = day_data.set_index('hour')[zones]
            if not zone_data.empty and not zone_data.isna().all().all():
                zone_data.plot(kind='bar', stacked=True, figsize=(12, 6), color=sns.color_palette('colorblind', 3))
                plt.title(f"Time in Zones (Cow {cow_id} - Day {date})")
                plt.xlabel('Hour')
                plt.ylabel('Time (hours)')
                plt.legend(['Eating', 'Sleeping', 'Null Zone'], title='Zone')
                plt.grid(True, linestyle='--', alpha=0.5)
                plt.xticks(rotation=45)
                plt.tight_layout()
                plt.savefig(f"{OUTPUT_DIR}/zone_time_cow_{cow_id}_day_{date}_{timestamp}.png")
                plt.close()
            if len(day_data['hour']) == len(day_data['net_displacement']) and not day_data['net_displacement'].isna().all():
                plt.figure(figsize=(12, 6))
                plt.plot(day_data['hour'], day_data['net_displacement'], marker='o', color='purple')
                plt.title(f"Net Displacement (Cow {cow_id} - Day {date})")
                plt.xlabel('Time')
                plt.ylabel('Displacement (meters)')
                plt.grid(True, linestyle='--', alpha=0.5)
                plt.xticks(rotation=45)
                plt.tight_layout()
                plt.savefig(f"{OUTPUT_DIR}/net_displacement_cow_{cow_id}_day_{date}_{timestamp}.png")
                plt.close()
            if len(day_data['hour']) == len(day_data['tortuosity']) and not day_data['tortuosity'].isna().all():
                plt.figure(figsize=(12, 6))
                plt.plot(day_data['hour'], day_data['tortuosity'], marker='o', color='brown')
                plt.title(f"Tortuosity (Cow {cow_id} - Day {date})")
                plt.xlabel('Time')
                plt.ylabel('Tortuosity')
                plt.grid(True, linestyle='--', alpha=0.5)
                plt.xticks(rotation=45)
                plt.tight_layout()
                plt.savefig(f"{OUTPUT_DIR}/tortuosity_cow_{cow_id}_day_{date}_{timestamp}.png")
                plt.close()
            if len(day_data['hour']) == len(day_data['num_stops']) and not day_data['num_stops'].isna().all():
                plt.figure(figsize=(12, 6))
                plt.bar(day_data['hour'], day_data['num_stops'], color='teal')
                plt.title(f"Number of Stops (Cow {cow_id} - Day {date})")
                plt.xlabel('Hour')
                plt.ylabel('Stops')
                plt.grid(True, linestyle='--', alpha=0.5)
                plt.xticks(rotation=45)
                plt.tight_layout()
                plt.savefig(f"{OUTPUT_DIR}/num_stops_cow_{cow_id}_day_{date}_{timestamp}.png")
                plt.close()
            if len(day_data['hour']) == len(day_data['avg_stop_duration_s']) and not day_data['avg_stop_duration_s'].isna().all():
                plt.figure(figsize=(12, 6))
                plt.plot(day_data['hour'], day_data['avg_stop_duration_s'], marker='o', color='cyan')
                plt.title(f"Average Stop Duration (Cow {cow_id} - Day {date})")
                plt.xlabel('Time')
                plt.ylabel('Duration (seconds)')
                plt.grid(True, linestyle='--', alpha=0.5)
                plt.xticks(rotation=45)
                plt.tight_layout()
                plt.savefig(f"{OUTPUT_DIR}/avg_stop_duration_cow_{cow_id}_day_{date}_{timestamp}.png")
                plt.close()
            if len(day_data['hour']) == len(day_data['resting_hours']) and not day_data['resting_hours'].isna().all():
                plt.figure(figsize=(12, 6))
                plt.plot(day_data['hour'], day_data['resting_hours'], marker='o', color='green')
                plt.title(f"Resting Hours (Cow {cow_id} - Day {date})")
                plt.xlabel('Time')
                plt.ylabel('Time (hours)')
                plt.grid(True, linestyle='--', alpha=0.5)
                plt.xticks(rotation=45)
                plt.tight_layout()
                plt.savefig(f"{OUTPUT_DIR}/resting_hours_cow_{cow_id}_day_{date}_{timestamp}.png")
                plt.close()
            if len(day_data['hour']) == len(day_data['zone_transitions']) and not day_data['zone_transitions'].isna().all():
                plt.figure(figsize=(12, 6))
                plt.bar(day_data['hour'], day_data['zone_transitions'], color='magenta')
                plt.title(f"Zone Transitions (Cow {cow_id} - Day {date})")
                plt.xlabel('Hour')
                plt.ylabel('Transitions')
                plt.grid(True, linestyle='--', alpha=0.5)
                plt.xticks(rotation=45)
                plt.tight_layout()
                plt.savefig(f"{OUTPUT_DIR}/zone_transitions_cow_{cow_id}_day_{date}_{timestamp}.png")
                plt.close()
            if len(day_data['hour']) == len(day_data['turning_angle_variance']) and not day_data['turning_angle_variance'].isna().all():
                plt.figure(figsize=(12, 6))
                plt.plot(day_data['hour'], day_data['turning_angle_variance'], marker='o', color='red')
                plt.title(f"Turning Angle Variance (Cow {cow_id} - Day {date})")
                plt.xlabel('Time')
                plt.ylabel('Variance (radians²)')
                plt.grid(True, linestyle='--', alpha=0.5)
                plt.xticks(rotation=45)
                plt.tight_layout()
                plt.savefig(f"{OUTPUT_DIR}/turning_variance_cow_{cow_id}_day_{date}_{timestamp}.png")
                plt.close()
            nearest_data = day_data.get('avg_nearest_neighbor_distance', pd.Series(np.nan))
            if len(day_data['hour']) == len(nearest_data) and not nearest_data.isna().all():
                plt.figure(figsize=(12, 6))
                plt.plot(day_data['hour'], nearest_data, marker='o', color='pink')
                plt.title(f"Average Nearest Neighbor Distance (Cow {cow_id} - Day {date})")
                plt.xlabel('Time')
                plt.ylabel('Distance (meters)')
                plt.grid(True, linestyle='--', alpha=0.5)
                plt.xticks(rotation=45)
                plt.tight_layout()
                plt.savefig(f"{OUTPUT_DIR}/nearest_neighbor_cow_{cow_id}_day_{date}_{timestamp}.png")
                plt.close()
        daily_data = cow_data.groupby('date').agg({
            'total_distance': 'sum',
            'avg_speed': 'mean',
            'time_eating_hours': 'sum',
            'time_sleeping_hours': 'sum',
            'time_null_zone_hours': 'sum',
            'net_displacement': 'mean',
            'tortuosity': 'mean',
            'num_stops': 'sum',
            'avg_stop_duration_s': 'mean',
            'resting_hours': 'sum',
            'turning_angle_variance': 'mean',
            'avg_nearest_neighbor_distance': 'mean'
        }).reset_index()
        if daily_data.empty:
            continue
        if len(daily_data['date']) == len(daily_data['total_distance']) and not daily_data['total_distance'].isna().all():
            plt.figure(figsize=(12, 6))
            plt.plot(daily_data['date'], daily_data['total_distance'], marker='o', color='blue')
            plt.title(f"Total Distance (Cow {cow_id} - Overall)")
            plt.xlabel('Date')
            plt.ylabel('Distance (meters)')
            plt.grid(True, linestyle='--', alpha=0.5)
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.savefig(f"{OUTPUT_DIR}/total_distance_cow_{cow_id}_overall_{timestamp}.png")
            plt.close()
        if len(daily_data['date']) == len(daily_data['avg_speed']) and not daily_data['avg_speed'].isna().all():
            plt.figure(figsize=(12, 6))
            plt.plot(daily_data['date'], daily_data['avg_speed'], marker='o', color='orange')
            plt.title(f"Average Speed (Cow {cow_id} - Overall)")
            plt.xlabel('Date')
            plt.ylabel('Speed (m/s)')
            plt.grid(True, linestyle='--', alpha=0.5)
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.savefig(f"{OUTPUT_DIR}/avg_speed_cow_{cow_id}_overall_{timestamp}.png")
            plt.close()
        zones = ['time_eating_hours', 'time_sleeping_hours', 'time_null_zone_hours']
        zone_data = daily_data.set_index('date')[zones]
        if not zone_data.empty and not zone_data.isna().all().all():
            zone_data.plot(kind='bar', stacked=True, figsize=(12, 6), color=sns.color_palette('colorblind', 3))
            plt.title(f"Time in Zones (Cow {cow_id} - Overall)")
            plt.xlabel('Date')
            plt.ylabel('Time (hours)')
            plt.legend(['Eating', 'Sleeping', 'Null Zone'], title='Zone')
            plt.grid(True, linestyle='--', alpha=0.5)
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.savefig(f"{OUTPUT_DIR}/zone_time_cow_{cow_id}_overall_{timestamp}.png")
            plt.close()
        if len(daily_data['date']) == len(daily_data['net_displacement']) and not daily_data['net_displacement'].isna().all():
            plt.figure(figsize=(12, 6))
            plt.plot(daily_data['date'], daily_data['net_displacement'], marker='o', color='purple')
            plt.title(f"Net Displacement (Cow {cow_id} - Overall)")
            plt.xlabel('Date')
            plt.ylabel('Displacement (meters)')
            plt.grid(True, linestyle='--', alpha=0.5)
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.savefig(f"{OUTPUT_DIR}/net_displacement_cow_{cow_id}_overall_{timestamp}.png")
            plt.close()
        if len(daily_data['date']) == len(daily_data['tortuosity']) and not daily_data['tortuosity'].isna().all():
            plt.figure(figsize=(12, 6))
            plt.plot(daily_data['date'], daily_data['tortuosity'], marker='o', color='brown')
            plt.title(f"Tortuosity (Cow {cow_id} - Overall)")
            plt.xlabel('Date')
            plt.ylabel('Tortuosity')
            plt.grid(True, linestyle='--', alpha=0.5)
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.savefig(f"{OUTPUT_DIR}/tortuosity_cow_{cow_id}_overall_{timestamp}.png")
            plt.close()
        if len(daily_data['date']) == len(daily_data['num_stops']) and not daily_data['num_stops'].isna().all():
            plt.figure(figsize=(12, 6))
            plt.bar(daily_data['date'], daily_data['num_stops'], color='teal')
            plt.title(f"Number of Stops (Cow {cow_id} - Overall)")
            plt.xlabel('Date')
            plt.ylabel('Stops')
            plt.grid(True, linestyle='--', alpha=0.5)
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.savefig(f"{OUTPUT_DIR}/num_stops_cow_{cow_id}_overall_{timestamp}.png")
            plt.close()
        if len(daily_data['date']) == len(daily_data['avg_stop_duration_s']) and not daily_data['avg_stop_duration_s'].isna().all():
            plt.figure(figsize=(12, 6))
            plt.plot(daily_data['date'], daily_data['avg_stop_duration_s'], marker='o', color='cyan')
            plt.title(f"Average Stop Duration (Cow {cow_id} - Overall)")
            plt.xlabel('Date')
            plt.ylabel('Duration (seconds)')
            plt.grid(True, linestyle='--', alpha=0.5)
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.savefig(f"{OUTPUT_DIR}/avg_stop_duration_cow_{cow_id}_overall_{timestamp}.png")
            plt.close()
        if len(daily_data['date']) == len(daily_data['resting_hours']) and not daily_data['resting_hours'].isna().all():
            plt.figure(figsize=(12, 6))
            plt.plot(daily_data['date'], daily_data['resting_hours'], marker='o', color='green')
            plt.title(f"Resting Hours (Cow {cow_id} - Overall)")
            plt.xlabel('Date')
            plt.ylabel('Time (hours)')
            plt.grid(True, linestyle='--', alpha=0.5)
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.savefig(f"{OUTPUT_DIR}/resting_hours_cow_{cow_id}_overall_{timestamp}.png")
            plt.close()
        if len(daily_data['date']) == len(daily_data['turning_angle_variance']) and not daily_data['turning_angle_variance'].isna().all():
            plt.figure(figsize=(12, 6))
            plt.plot(daily_data['date'], daily_data['turning_angle_variance'], marker='o', color='red')
            plt.title(f"Turning Angle Variance (Cow {cow_id} - Overall)")
            plt.xlabel('Date')
            plt.ylabel('Variance (radians²)')
            plt.grid(True, linestyle='--', alpha=0.5)
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.savefig(f"{OUTPUT_DIR}/turning_variance_cow_{cow_id}_overall_{timestamp}.png")
            plt.close()
        if len(daily_data['date']) == len(daily_data['avg_nearest_neighbor_distance']) and not daily_data['avg_nearest_neighbor_distance'].isna().all():
            plt.figure(figsize=(12, 6))
            plt.plot(daily_data['date'], daily_data['avg_nearest_neighbor_distance'], marker='o', color='pink')
            plt.title(f"Average Nearest Neighbor Distance (Cow {cow_id} - Overall)")
            plt.xlabel('Date')
            plt.ylabel('Distance (meters)')
            plt.grid(True, linestyle='--', alpha=0.5)
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.savefig(f"{OUTPUT_DIR}/nearest_neighbor_cow_{cow_id}_overall_{timestamp}.png")
            plt.close()

# New Plotting Functions
def speed_distribution_histograms(df, timestamp):
    if df.empty:
        return
    plt.figure(figsize=(10, 6))
    sns.histplot(df['speed'], bins=30, color='blue', kde=True)
    plt.title("Speed Distribution - All Cows")
    plt.xlabel('Speed (m/s)')
    plt.ylabel('Frequency')
    plt.savefig(f"{OUTPUT_DIR}/speed_distribution_all_{timestamp}.png")
    plt.close()
    for cow_id in df['id'].unique():
        cow_data = df[df['id'] == cow_id]
        if cow_data.empty:
            continue
        plt.figure(figsize=(10, 6))
        sns.histplot(cow_data['speed'], bins=30, color='blue', kde=True)
        plt.title(f"Speed Distribution (Cow {cow_id})")
        plt.xlabel('Speed (m/s)')
        plt.ylabel('Frequency')
        plt.savefig(f"{OUTPUT_DIR}/speed_distribution_cow_{cow_id}_{timestamp}.png")
        plt.close()

def zone_occupancy_over_time(historical_metrics_df, timestamp):
    if historical_metrics_df.empty:
        return
    zones = ['time_eating_hours', 'time_sleeping_hours', 'time_null_zone_hours']
    for zone in zones:
        if zone not in historical_metrics_df.columns:
            continue
        plt.figure(figsize=(12, 6))
        for cow_id in historical_metrics_df['cow_id'].unique():
            cow_data = historical_metrics_df[historical_metrics_df['cow_id'] == cow_id]
            if cow_data.empty:
                continue
            plt.plot(cow_data['hour'], cow_data[zone], label=f"Cow {cow_id}")
        plt.title(f"{zone.replace('_', ' ').title()} Over Time")
        plt.xlabel('Time')
        plt.ylabel('Time (hours)')
        plt.legend()
        plt.grid(True)
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig(f"{OUTPUT_DIR}/zone_occupancy_{zone}_{timestamp}.png")
        plt.close()

def correlation_heatmaps(historical_metrics_df, timestamp):
    if historical_metrics_df.empty:
        return
    metrics = ['total_distance', 'avg_speed', 'time_eating_hours', 'time_sleeping_hours', 'time_null_zone_hours', 'num_stops']
    corr_data = historical_metrics_df[metrics].corr()
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr_data, annot=True, cmap='coolwarm', fmt='.2f')
    plt.title("Correlation Heatmap of Metrics")
    plt.savefig(f"{OUTPUT_DIR}/correlation_heatmap_{timestamp}.png")
    plt.close()

def box_plots_for_metrics(snapshot_df, timestamp):
    if snapshot_df.empty:
        return
    metrics = ['hourly_total_distance', 'hourly_avg_speed', 'hourly_num_stops']
    for metric in metrics:
        if metric not in snapshot_df.columns:
            continue
        plt.figure(figsize=(10, 6))
        sns.boxplot(x='cow_id', y=metric, data=snapshot_df)
        plt.title(f"Box Plot of {metric.replace('_', ' ').title()}")
        plt.xlabel('Cow ID')
        plt.ylabel(metric.replace('_', ' ').title())
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig(f"{OUTPUT_DIR}/boxplot_{metric}_{timestamp}.png")
        plt.close()

def pair_plots_for_clustering(historical_metrics_df, timestamp):
    if historical_metrics_df.empty:
        return
    metrics = ['total_distance', 'avg_speed', 'time_eating_hours']
    if all(metric in historical_metrics_df.columns for metric in metrics):
        sns.pairplot(historical_metrics_df[metrics + ['cow_id']], hue='cow_id')
        plt.suptitle("Pair Plot for Clustering", y=1.02)
        plt.savefig(f"{OUTPUT_DIR}/pairplot_clustering_{timestamp}.png")
        plt.close()

def time_series_zone_transitions(historical_metrics_df, timestamp):
    if historical_metrics_df.empty:
        return
    if 'zone_transitions' not in historical_metrics_df.columns:
        return
    plt.figure(figsize=(12, 6))
    for cow_id in historical_metrics_df['cow_id'].unique():
        cow_data = historical_metrics_df[historical_metrics_df['cow_id'] == cow_id]
        if cow_data.empty:
            continue
        plt.plot(cow_data['hour'], cow_data['zone_transitions'], label=f"Cow {cow_id}")
    plt.title("Zone Transitions Over Time")
    plt.xlabel('Time')
    plt.ylabel('Number of Transitions')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(f"{OUTPUT_DIR}/zone_transitions_over_time_{timestamp}.png")
    plt.close()

def polar_plots_turning_angles(df, timestamp):
    if df.empty:
        return
    def compute_turning_angles(sub):
        dx = sub['x_smooth'].diff()
        dy = sub['y_smooth'].diff()
        angles = np.arctan2(dy, dx)
        turning_angles = angles.diff().abs().dropna()
        return turning_angles
    all_angles = []
    for cow_id in df['id'].unique():
        cow_data = df[df['id'] == cow_id]
        if len(cow_data) < 3:
            continue
        all_angles.extend(compute_turning_angles(cow_data).tolist())
    if all_angles:
        plt.figure(figsize=(8, 8))
        ax = plt.subplot(111, polar=True)
        ax.hist(all_angles, bins=30, color='teal', alpha=0.7)
        ax.set_title("Polar Plot of Turning Angles - All Cows")
        plt.savefig(f"{OUTPUT_DIR}/polar_turning_angles_all_{timestamp}.png")
        plt.close()
    for cow_id in df['id'].unique():
        cow_data = df[df['id'] == cow_id]
        if len(cow_data) < 3:
            continue
        angles = compute_turning_angles(cow_data)
        if not angles.empty:
            plt.figure(figsize=(8, 8))
            ax = plt.subplot(111, polar=True)
            ax.hist(angles, bins=30, color='teal', alpha=0.7)
            ax.set_title(f"Polar Plot of Turning Angles (Cow {cow_id})")
            plt.savefig(f"{OUTPUT_DIR}/polar_turning_angles_cow_{cow_id}_{timestamp}.png")
            plt.close()

def heatmaps_cow_interactions(snapshot_df, timestamp):
    if snapshot_df.empty or 'hourly_avg_nearest_neighbor_distance' not in snapshot_df.columns:
        return
    pivot_data = snapshot_df.pivot(index='timestamp', columns='cow_id', values='hourly_avg_nearest_neighbor_distance')
    if pivot_data.empty:
        return
    plt.figure(figsize=(12, 8))
    sns.heatmap(pivot_data, annot=True, cmap='viridis', fmt='.2f')
    plt.title("Heatmap of Average Nearest Neighbor Distances")
    plt.xlabel('Cow ID')
    plt.ylabel('Timestamp')
    plt.savefig(f"{OUTPUT_DIR}/interaction_heatmap_{timestamp}.png")
    plt.close()

def anomaly_detection_plots(snapshot_df, timestamp):
    if snapshot_df.empty:
        return
    metrics = ['hourly_avg_speed', 'hourly_total_distance']
    for metric in metrics:
        if metric not in snapshot_df.columns:
            continue
        mean_val = snapshot_df[metric].mean()
        std_val = snapshot_df[metric].std()
        threshold_high = mean_val + 2 * std_val
        threshold_low = mean_val - 2 * std_val
        plt.figure(figsize=(12, 6))
        plt.plot(snapshot_df['cow_id'], snapshot_df[metric], marker='o', label=metric)
        plt.axhline(y=threshold_high, color='r', linestyle='--', label='High Threshold')
        plt.axhline(y=threshold_low, color='r', linestyle='--', label='Low Threshold')
        plt.title(f"Anomaly Detection for {metric.replace('_', ' ').title()}")
        plt.xlabel('Cow ID')
        plt.ylabel(metric.replace('_', ' ').title())
        plt.legend()
        plt.grid(True)
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig(f"{OUTPUT_DIR}/anomaly_{metric}_{timestamp}.png")
        plt.close()

def cumulative_distance_plots(df, timestamp):
    if df.empty:
        return
    for cow_id in df['id'].unique():
        cow_data = df[df['id'] == cow_id].sort_values('time')
        if cow_data.empty:
            continue
        cow_data['cumulative_distance'] = cow_data['distance_meters'].cumsum()
        plt.figure(figsize=(12, 6))
        plt.plot(cow_data['time'], cow_data['cumulative_distance'], color='blue')
        plt.title(f"Cumulative Distance (Cow {cow_id})")
        plt.xlabel('Time')
        plt.ylabel('Cumulative Distance (meters)')
        plt.grid(True)
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig(f"{OUTPUT_DIR}/cumulative_distance_cow_{cow_id}_{timestamp}.png")
        plt.close()

def stop_duration_histograms(df, timestamp):
    if df.empty:
        return
    stop_data = df[df['speed'] < STOP_SPEED_THRESHOLD]
    if stop_data.empty:
        return
    plt.figure(figsize=(10, 6))
    sns.histplot(stop_data['time_diff'], bins=30, color='purple', kde=True)
    plt.title("Stop Duration Distribution - All Cows")
    plt.xlabel('Stop Duration (seconds)')
    plt.ylabel('Frequency')
    plt.savefig(f"{OUTPUT_DIR}/stop_duration_all_{timestamp}.png")
    plt.close()
    for cow_id in stop_data['id'].unique():
        cow_stop_data = stop_data[stop_data['id'] == cow_id]
        if cow_stop_data.empty:
            continue
        plt.figure(figsize=(10, 6))
        sns.histplot(cow_stop_data['time_diff'], bins=30, color='purple', kde=True)
        plt.title(f"Stop Duration Distribution (Cow {cow_id})")
        plt.xlabel('Stop Duration (seconds)')
        plt.ylabel('Frequency')
        plt.savefig(f"{OUTPUT_DIR}/stop_duration_cow_{cow_id}_{timestamp}.png")
        plt.close()

def activity_vs_time_of_day(df, timestamp):
    if df.empty:
        return
    df['hour_of_day'] = df['time'].dt.hour
    metrics = ['speed', 'distance_meters']
    for metric in metrics:
        if metric not in df.columns:
            continue
        grouped = df.groupby('hour_of_day')[metric].mean()
        plt.figure(figsize=(12, 6))
        plt.plot(grouped.index, grouped.values, marker='o', color='green')
        plt.title(f"Average {metric.replace('_', ' ').title()} vs. Time of Day")
        plt.xlabel('Hour of Day')
        plt.ylabel(f"Average {metric.replace('_', ' ').title()}")
        plt.grid(True)
        plt.xticks(range(0, 24))
        plt.tight_layout()
        plt.savefig(f"{OUTPUT_DIR}/activity_vs_time_{metric}_{timestamp}.png")
        plt.close()

def additional_analysis(df, timestamp):
    if df.empty:
        return
    historical_metrics_df = get_historical_metrics_df()
    spatial_heatmap(df, timestamp, by_day=True)
    spatial_heatmap(df, timestamp, by_day=False)
    zone_transition_matrix(df, timestamp, by_day=True)
    zone_transition_matrix(df, timestamp, by_day=False)
    temporal_activity_profiles(df, timestamp, by_day=True)
    temporal_activity_profiles(df, timestamp, by_day=False)
    behavioral_clustering(df, timestamp, by_day=True)
    behavioral_clustering(df, timestamp, by_day=False)
    trajectory_analysis(df, timestamp, by_day=True)
    trajectory_analysis(df, timestamp, by_day=False)
    per_cow_detailed_time_series(df, timestamp, by_day=True)
    per_cow_detailed_time_series(df, timestamp, by_day=False)
    per_cow_zone_usage(df, timestamp)
    per_cow_resting_analysis(df, aggregate_time(df), timestamp, by_day=True)
    per_cow_resting_analysis(df, aggregate_time(df), timestamp, by_day=False)
    per_cow_turning_analysis(df, timestamp, by_day=True)
    per_cow_turning_analysis(df, timestamp, by_day=False)
    per_cow_path_trajectory(df, timestamp, by_day=True)
    per_cow_path_trajectory(df, timestamp, by_day=False)
    per_cow_advanced_metrics(df, timestamp)
    speed_distribution_histograms(df, timestamp)
    zone_occupancy_over_time(historical_metrics_df, timestamp)
    correlation_heatmaps(historical_metrics_df, timestamp)
    box_plots_for_metrics(latest_snapshot, timestamp)
    pair_plots_for_clustering(historical_metrics_df, timestamp)
    time_series_zone_transitions(historical_metrics_df, timestamp)
    polar_plots_turning_angles(df, timestamp)
    heatmaps_cow_interactions(latest_snapshot, timestamp)
    anomaly_detection_plots(latest_snapshot, timestamp)
    cumulative_distance_plots(df, timestamp)
    stop_duration_histograms(df, timestamp)
    activity_vs_time_of_day(df, timestamp)

def snapshot_computation_thread():
    global last_upload_time
    while not stop_flag:
        start_time = datetime.utcnow().timestamp()
        if snapshot_compute_lock.acquire(blocking=False):
            try:
                timestamp = datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')
                with data_lock:
                    current_df_copy = current_df.copy() if not current_df.empty else pd.DataFrame()
                    valid_ids_copy = load_valid_ids()
                if current_df_copy.empty:
                    print(f"Skipping snapshot computation at {timestamp}: No new data")
                else:
                    snapshot_df = compute_activity_snapshot(current_df_copy, timestamp, valid_ids_copy)
                    current_time = datetime.utcnow().timestamp()
                    time_diff = (current_time - last_upload_time) if last_upload_time else float('inf')
                    print(f"Checking upload at {timestamp}: diff={time_diff:.0f}s, threshold={UPLOAD_INTERVAL_SEC}s")

                    if last_upload_time is None or time_diff >= UPLOAD_INTERVAL_SEC:
                        # 1) upload Firestore snapshot
                        upload_activity_snapshot(snapshot_df, timestamp)
                        # 2) generate & save all plots
                        additional_analysis(current_df_copy, timestamp)
                        # 3) upload saved plots
                        upload_plots_to_storage(timestamp)
                        # 4) print summary & clear local files
                        print_analysis_summary(current_df_copy, snapshot_df, timestamp)
                        clear_plots_folder()
                        last_upload_time = current_time
                    else:
                        print(f"Upload skipped at {timestamp}: only {time_diff:.0f}s since last upload")
            finally:
                snapshot_compute_lock.release()

        # sleep until next compute
        elapsed = datetime.utcnow().timestamp() - start_time
        time.sleep(max(0, SNAPSHOT_COMPUTE_INTERVAL_SEC - elapsed))


def on_snapshot(doc_snapshot, changes, read_time):
    for change in changes:
        if change.type.name in ['ADDED', 'MODIFIED']:
            doc = change.document
            data = doc.to_dict()
            ts = data.get('timestamp')
            if not ts:
                continue
            cow_id = doc.reference.parent.parent.id
            eid = f"{cow_id}_{ts}"
            if eid in processed_timestamps:
                continue
            data['id'] = cow_id
            data_queue.append(data)
            processed_timestamps.add(eid)

def setup_listeners(valid_ids):
    global listeners
    for listener in listeners:
        try:
            listener.unsubscribe()
        except Exception as e:
            print(f"Error unsubscribing listener: {e}")
    listeners.clear()
    for cow_id in valid_ids:
        coords_ref = db.collection('cows').document(cow_id).collection('coordinates')
        listener = coords_ref.on_snapshot(on_snapshot)
        listeners.append(listener)
    print(f"Set up listeners for {len(valid_ids)} cows")

def process_data():
    global last_analysis_time, last_upload_time, current_df, historical_df, last_processed_timestamps

    init_processed_timestamps()
    init_historical_metrics()

    # Load or fetch valid IDs
    valid_ids = load_valid_ids()
    initial_df, initial_valid_ids = fetch_initial_data()
    if not valid_ids:
        valid_ids = initial_valid_ids
        save_valid_ids(valid_ids)

    setup_listeners(valid_ids)

    # Process the initial batch
    if not initial_df.empty:
        batch_data = initial_df.to_dict('records')
        while batch_data:
            batch = batch_data[:BATCH_SIZE]
            batch_data = batch_data[BATCH_SIZE:]
            batch_df = pd.DataFrame(batch)
            processed_batch = preprocess_data(batch_df)
            if not processed_batch.empty:
                with data_lock:
                    current_df = pd.concat([current_df, processed_batch]) if not current_df.empty else processed_batch
                    historical_df = pd.concat([historical_df, processed_batch]) if not historical_df.empty else processed_batch
                    for cow_id in processed_batch['id'].unique():
                        max_ts = processed_batch[processed_batch['id'] == cow_id]['time'].max()
                        if pd.notna(max_ts):
                            last_processed_timestamps[cow_id] = max_ts
                save_processed_data(processed_batch)
                update_historical_metrics(
                    processed_batch,
                    pd.to_datetime(datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ'), utc=True)
                )

        save_processed_timestamps()
        save_historical_metrics()

        print(f"Initial processing complete. current_df has {len(current_df)} rows")

        # Immediate first run snapshot and upload
        if not current_df.empty:
            timestamp = datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')
            with snapshot_compute_lock:
                with data_lock:
                    current_df_copy = current_df.copy()
                    valid_ids_copy = valid_ids.copy()

                # 1) compute snapshot
                snapshot_df = compute_activity_snapshot(current_df_copy, timestamp, valid_ids_copy)
                # 2) generate & save all plots
                additional_analysis(current_df_copy, timestamp)
                # 3) upload Firestore snapshot
                upload_activity_snapshot(snapshot_df, timestamp)
                # 4) upload the plots you just generated
                upload_plots_to_storage(timestamp)
                # 5) print summary & clear disk
                print_analysis_summary(current_df_copy, snapshot_df, timestamp)
                clear_plots_folder()

            last_upload_time = datetime.utcnow().timestamp()
            last_analysis_time = last_upload_time
        else:
            print("Skipping first analysis because current_df is empty")
            last_upload_time = datetime.utcnow().timestamp()
            last_analysis_time = last_upload_time

    else:
        print("No initial data to process; skipping first analysis")
        last_upload_time = datetime.utcnow().timestamp()
        last_analysis_time = last_upload_time

    # Start the background snapshot thread
    snapshot_thread = threading.Thread(target=snapshot_computation_thread, daemon=True)
    snapshot_thread.start()

    # Main loop: ingest live updates
    while not stop_flag:
        new_records = 0
        while data_queue:
            batch = data_queue[:BATCH_SIZE]
            data_queue[:BATCH_SIZE] = []
            batch_df = pd.DataFrame(batch)
            processed_batch = preprocess_data(batch_df)
            if not processed_batch.empty:
                new_records += len(processed_batch)
                with data_lock:
                    current_df = pd.concat([current_df, processed_batch]) if not current_df.empty else processed_batch
                    historical_df = pd.concat([historical_df, processed_batch]) if not historical_df.empty else processed_batch
                    for cow_id in processed_batch['id'].unique():
                        max_ts = processed_batch[processed_batch['id'] == cow_id]['time'].max()
                        if pd.notna(max_ts):
                            last_processed_timestamps[cow_id] = max_ts
                save_processed_data(processed_batch)
                update_historical_metrics(
                    processed_batch,
                    pd.to_datetime(datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ'), utc=True)
                )
            save_processed_timestamps()

        if new_records:
            print(f"Processed {new_records} new records in this cycle")

        valid_ids = update_valid_cow_ids()
        time.sleep(MINIMAL_LOOP_SLEEP)

if __name__ == "__main__":
    try:
        process_data()
    except KeyboardInterrupt:
        print("Shutting down...")
        stop_flag = True
        save_processed_timestamps()
        save_historical_metrics()
        for listener in listeners:
            try:
                listener.unsubscribe()
            except Exception as e:
                print(f"Error unsubscribing listener: {e}")
        listeners.clear()

Strømmer utdata som er avkortet til de siste 5000 linjene.
  Turning Angle Variance: 2.59
  Average Nearest Neighbor Distance: 0.02 meters
Historical Metrics:
  Historical Average Speed: 0.03 m/s
  Historical Average Total Distance per Hour: 20.72 meters/hour
  Historical Average Time in Eating Zone: 0.10 hours
  Historical Average Time in Sleeping Zone: 0.00 hours
  Historical Average Time in Null Zone: 0.06 hours
  Historical Average Net Displacement: 2.34 meters
  Historical Average Tortuosity: 8.80
  Historical Average Number of Stops: 13.62
  Historical Average Stop Duration: 2.31 seconds
  Historical Average Resting Hours: 0.12 hours
  Historical Average Zone Transitions: 7.75
  Historical Average Turning Angle Variance: 2.28
  Historical Total Distance: 82.87 meters
  Historical Total Number of Stops: 54.5
  Historical Total Zone Transitions: 31
  Historical Total Time in Eating Zone: 0.38 hours
  Historical Total Time in Sleeping Zone: 0.00 hours
  Historical Total Time in Null